# Assignment 2 : Language Modeling & Evaluation 

**Student names**: Vegard Aa Albretsen <br>
**Group number**: Group 62 <br>
**Date**: _We will see_

## Important notes
Please read and follow these rules. Submissions that do not fulfill them may be returned.
1. You may work in groups of maximum 2 students.
2. Submit in **.ipynb** format only.
3. The assignment must be typed. Handwritten answers are not accepted.

**Due date**: 28.09.2025 23:59

### What you will do
- Build a **unigram document language model** with a **document-term matrix**.
- Rank documents for queries using **Jelinek-Mercer smoothing**.
- Evaluate the run using **Cranfield queries and qrels** (P@k, MAP, MRR).
- (Optional) Try **Dirichlet** and compare briefly.


---
## Dataset

Make sure the Cranfield files are placed next to the notebook:
- `cran.all.1400` — document collection (1400 docs)
- `cran.qry` — queries
- `cranqrel` — relevance judgments (qrels)

> Only the **document parsing** for cran.all.1400 code is provided below. You will implement the rest in the TODO cells.


### Load and parse documents (provided)

Run the cell to parse the Cranfield documents. Update the path so it points to your `cran.all.1400` file.


In [22]:
# TODO: Read 'cran.all.1400' and parse the documents into a suitable data structure

CRAN_PATH = r"cran.all.1400"  # <-- change this!

def parse_cranfield(path):
    docs = {} # docs : Dict[int, Dict[str, Union[int, str]]] -> {id: {id, title, abstract}}
    current_id = None
    current_field = None
    buffers = {"T": [], "A": [], "B": [], "W": []}
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.rstrip("\n")
            if line.startswith(".I "):
                if current_id is not None:
                    docs[current_id] = {
                        "id": current_id,
                        "title": " ".join(buffers["T"]).strip(),
                        "abstract": " ".join(buffers["W"]).strip()
                    }
                current_id = int(line.split()[1])
                buffers = {k: [] for k in buffers}
                current_field = None
            elif line.startswith("."):
                tag = line[1:].strip()
                current_field = tag if tag in buffers else None
            else:
                if current_field is not None:
                    buffers[current_field].append(line)
    if current_id is not None:
        docs[current_id] = {
            "id": current_id,
            "title": " ".join(buffers["T"]).strip(),
            "abstract": " ".join(buffers["W"]).strip()
        }
    print(f"Parsed {len(docs)} documents.")
    return docs

docs = parse_cranfield(CRAN_PATH)
print(list(docs.items())[:1])  # peek at the first parsed doc


Parsed 1400 documents.
[(1, {'id': 1, 'title': 'experimental investigation of the aerodynamics of a wing in a slipstream .', 'abstract': 'experimental investigation of the aerodynamics of a wing in a slipstream .   an experimental study of a wing in a propeller slipstream was made in order to determine the spanwise distribution of the lift increase due to slipstream at different angles of attack of the wing and at different free stream to slipstream velocity ratios .  the results were intended in part as an evaluation basis for different theoretical treatments of this problem .   the comparative span loading curves, together with supporting evidence, showed that a substantial part of the lift increment produced by the slipstream was due to a /destalling/ or boundary-layer-control effect .  the integrated remaining lift increment, after subtracting this destalling lift, was found to agree well with a potential flow theory .   an empirical evaluation of the destalling effects was made fo

## 2.1 Language Modeling

You will create a **unigram language model** per document, using a **document-term matrix**, and score queries with **Jelinek-Mercer smoothing**.


### 2.1.1 Preprocessing

Implement a simple tokenizer/normalizer (e.g., lowercasing, punctuation removal and stopword removal) and apply it to each document

- Return a list of tokens for each document.


In [23]:
# TODO: Implement preprocessing and apply to all documents

STOPWORDS = set("""a about above after again against all am an and any are aren't as at be because been
before being below between both but by can't cannot could couldn't did didn't do does doesn't doing don't down
during each few for from further had hadn't has hasn't have haven't having he he'd he'll he's her here here's hers
herself him himself his how how's i i'd i'll i'm i've if in into is isn't it it's its itself let's me more most
mustn't my myself no nor not of off on once only or other ought our ours ourselves out over own same shan't she
she'd she'll she's should shouldn't so some such than that that's the their theirs them themselves then there there's
these they they'd they'll they're they've this those through to too under until up very was wasn't we we'd we'll we're
we've were weren't what what's when when's where where's which while who who's whom why why's with won't would wouldn't
you you'd you'll you're you've your yours yourself yourselves""".split())

import re

def preprocess(text): 
    text = text.lower()
    tokens = re.findall(r'\b[a-zA-Z]+\b', text)
    tokens = [token for token in tokens if token not in STOPWORDS]
    return tokens

for doc_id, doc_data in docs.items():
    combined_text = doc_data['title'] + " " + doc_data['abstract']
    doc_data['tokens'] = preprocess(combined_text)


print(f"Example document tokens: {list(docs.items())[0][1]['tokens'][:10]}")
print(f"Document 1 has {len(docs[1]['tokens'])} tokens")


Example document tokens: ['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', 'experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream']
Document 1 has 84 tokens


### 2.1.2 Build the matrix

Construct:
- A vocabulary `term -> column_index`
- A (sparse) **document–term count matrix**
- Document lengths `|d|` and collection (all documents) totals

> Tip: You may use dictionaries or `scipy.sparse` for efficiency.


In [ ]:
# TODO: Build vocabulary, doc-term counts (sparse), document lengths, and collection totals
from collections import defaultdict
vocabulary = set()
doc_term_counts = {'term':str,'count':int}
for doc_id, doc_data in docs.items():
    for token in doc_data['tokens']:
        vocabulary.add(token)

vocab_to_idx = {term: idx for idx, term in enumerate(vocabulary)}

doc_term_counts = {}
doc_lengths = {}
collection_totals = defaultdict(int)

for doc_id, doc_data in docs.items():
    term_counts = defaultdict(int)
    for token in doc_data['tokens']:
        term_counts[token] += 1
        collection_totals[token] +=1
    
    doc_term_counts[doc_id] = dict(term_counts)
    doc_lengths[doc_id] = len(doc_data['tokens'])

print(f"Vocabulary size: {len(vocabulary)}")
print(f"Total documents: {len(doc_term_counts)}")
print(f"Document 1 length: {doc_lengths[1]}")
print(f"Total collection tokens: {sum(collection_totals.values())}")

6928
{'er', 'disregarding', 'currently', 'objects', 'denoted', 'quantitative', 'noticed', 'retained', 'duplicating', 'accumulation', 'cabin', 'ministry', 'boattailing', 'station', 'morning', 'interrelation', 'surface', 'arm', 'statistically', 'lags', 'unlikely', 'disequilibrium', 'conditions', 'published', 'arise', 'simplified', 'opposition', 'implement', 'able', 'incremental', 'heating', 'supports', 'receiver', 'implication', 'fabrication', 'sensitive', 'aluminium', 'influence', 'reaction', 'constructed', 'modify', 'warranted', 'negatively', 'untested', 'unified', 'step', 'pitch', 'flaps', 'aeroplane', 'size', 'first', 'six', 'modeled', 'suppress', 'reactive', 'situation', 'reflecting', 'maintain', 'collaboration', 'minimised', 'flex', 'merged', 'source', 'capabilities', 'deflexion', 'constituents', 'chloride', 'location', 'heterogeneous', 'stators', 'embraces', 'venus', 'winny', 'words', 'illustrations', 'nondimensional', 'slender', 'rose', 'intended', 'careful', 'grain', 'compressio

### 2.1.3 Rank with **Jelinek-Mercer smoothing**

Implement query likelihood scoring with Jelinek-Mercer smoothing:

$\hat{P}(t \mid M_d) = \lambda \hat{P}_{\text{mle}}(t \mid M_d) + (1 - \lambda)\hat{P}_{\text{mle}}(t \mid M_c), \ \lambda = 0.5$



In [ ]:
""" TODO: Create a function for implementing query likelihood scoring with Jelinek Mercer smoothing (λ=0.5), 
using the formula above."""

def jelinek_smoothing(query: str):
    # Your code here
    raise NotImplementedError("Implement Jelinek-Mercer smoothing here")

In [ ]:
# Do not change this code
queries_assignment2 = [
  "gas pressure",
  "structural aeroelastic flight high speed aircraft",
  "heat conduction composite slabs",
  "boundary layer control",
  "compressible flow nozzle",
  "combustion chamber injection",
  "laminar turbulent transition",
  "fatigue crack growth",
  "wing tip vortices",
  "propulsion efficiency"
]

In [ ]:
# Run Jelinek-Mercer smoothing on queries in batch (print top-10 results for each), using the function you created
def run_batch_smoothing(queries):
    results = {}
    for i, q in enumerate(queries, 1):
        res = jelinek_smoothing(q)
        results[f"Q{i}"] = res
    return results

jelinek_results = run_batch_smoothing(queries_assignment2)

for qid, res in jelinek_results.items():
    print(qid, "=>", res[:10])


#### Dirichlet

If you have time, also implement Dirichlet smoothing and briefly compare the top-10 lists for the first query in the queries_assignment2 list


In [ ]:
# TODO (Optional): Implement Dirichlet scoring and compare with Jelinek-Mercer
# Your code here


## 2.2 Evaluation (Cranfield queries + qrels)

Evaluate your retrieval system using **Cranfield**:
- Parse **queries** from `cran.qry`
- Parse **relevance judgments** from `cranqrel`
- Compute **P@k (k=5,10)**, **MAP (Mean Average Precision)**, and **MRR (Mean Reciprocal Rank)** over all queries


### 2.2.1 Parse `cran.qry` and `cranqrel`

- Create `queries[qid] = "text"` by parsing `cran.qry`
- Create `qrels[qid] = set(relevant_doc_ids)` by parsing `cranqrel`


In [ ]:
# TODO: Parse cran.qry and cranqrel into convenient data structures
# Your code here


### 2.2.2 Implement metrics: P@k, MAP, MRR

Write functions to compute:
- Precision@k (for **k=5** and **k=10**)
- Mean Average Precision (MAP)
- Mean Reciprocal Rank (MRR)


In [ ]:
# TODO: Implement P@k (k=5,10), MAP, and MRR
# Your code here


### 2.2.3 Evaluate your run

- For **all queries**, generate rankings with your **Jelinek-Mercer** model
- Report aggregate metrics: P@5, P@10, MAP, MRR


In [ ]:
# TODO: Run retrieval for all queries and compute P@5, P@10, MAP, and MRR
# Your code here




### 2.2.4 Interpolated precision–recall curves (11‑point)

- For **all queries**, if you don’t have query IDs, assign sequential IDs: `Q1, Q2, ..., Qm` in the order they appear.
- Using your **rankings from task 2.2.3** and the **relevance judgments (`cranqrel`)**, compute **precision** and **recall** at each rank for each query.
- For the 11 standard recall levels `R = {0.0, 0.1, ..., 1.0}`, compute the **interpolated precision** at level `r` as the **maximum precision** observed at any point with recall ≥ `r`.
- **Report/plot** the **11‑point interpolated precision–recall curve** across queries (and optionally a few per‑query curves).

In [ ]:

# TODO: Compute the 11-point interpolated precision–recall for each query

# Your code here
